<a href="https://colab.research.google.com/github/GGMmattos/Machine-Learning-And-Data-Science/blob/main/Taxi_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entendimento do ambiente

In [21]:
import gym
import random

In [ ]:
env = gym.make('Taxi-v3').env

In [23]:
env.reset()

368

In [ ]:
env.render()

Seis ações possíveis que o taxi pode fazer

In [25]:
print(env.action_space)

Discrete(6)


0: mova-se para o sul

1: mova-se para o norte

2: mova-se para o leste

3: mova-se para oeste

4: passageiro de coleta

5: deixar o passageiro

In [26]:
print(env.observation_space )

Discrete(500)


# Treinamento

In [31]:
random.uniform(0, 1)

0.8356880416528426

In [ ]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

In [36]:
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(100000):
  estado = env.reset()

  penalidades, recompensa = 0, 0
  done = False
  while not done:
    # Exploração
    if random.uniform(0, 1) < epsilon:
      acao = env.action_space.sample()
    # Explotation
    else:
      acao = np.argmax(q_table[estado])

    proximo_estado, recompensa, done, info = env.step(acao)

    q_antigo = q_table[estado, acao]
    proximo_maximo = np.max(q_table[proximo_estado])

    q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
    q_table[estado, acao] = q_novo

    if recompensa == -10:
      penalidades += 1

    estado = proximo_estado

  if i % 100 == 0:
    clear_output(wait=True)
    print('Episodio:', i)

print('Treinamento concluído')



Episodio: 99900
Treinamento concluído


# Avaliação

In [45]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
  estado = env.reset()
  penalidades, recompensa = 0, 0
  done = False
  while not done:
    acao = np.argmax(q_table[estado])
    estado, recompensa, done, info = env.step(acao)

    if recompensa == -10:
      penalidades += 1

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': estado,
        'action': acao,
        'reward': recompensa
    })
  total_penalidades += penalidades

print("Episódios", episodios)
print("Penalidades", total_penalidades)

Episódios 50
Penalidades 0


In [47]:
frames[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'frame': '+---------+\n|R: | : :G|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |\x1b\x1bB\x1b\x1b: |\n+---------+\n  (West)\n',
 'state': 471,
 'action': 3,
 'reward': -1}

In [49]:
from time import sleep
for frame in frames:
  clear_output(wait=True)
  print(frame['frame'])
  print('Estado', frame['state'])
  print('Ação', frame['action'])
  print('Recompensa', frame['reward'])
  sleep(.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Estado 410
Ação 5
Recompensa 20
